### Machine Learning 7, 8, 9, 10 and 11. Assignment 1

##### Problem Statement: Machine Learning 7
In this assignment, students will be using the K-Nearest neighbors algorithm to predict how many points NBA players scored in the 2013-2014 season. 

A look at the data. 

Before we dive into the algorithm, let's take a look at our data. Each row in the data contains information on how a player performed in the 2013-2014 NBA season. 

Download 'nba_2013.csv' file from this link: https://www.dropbox.com/s/b3nv38jjo5dxcl6/nba_2013.csv?dl=0
Hehre are some selected columns from the data:


Here are some selected columns from the data:

player - name of the player

pos - the position of the player

g - number of games the player was in

gs - number of games the player started

pts - total points the player scored

There are many more columns in the data, mostly containing information about average player game performance over the course of the season. 

In [1]:
import pandas as pd
import math

pd.set_option('display.max_columns', None)
with open("nba_2013.csv", 'r') as csvfile:
    nba = pd.read_csv(csvfile)

print(nba.columns.values) # The names of all the columns in the data.
nba.head()

['player' 'pos' 'age' 'bref_team_id' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p'
 'x3pa' 'x3p.' 'x2p' 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb'
 'trb' 'ast' 'stl' 'blk' 'tov' 'pf' 'pts' 'season' 'season_end']


,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,x2p,x2pa,x2p.,efg.,ft,fta,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,4,15,0.266667,62,126,0.492063,0.482,35,53,0.660,72,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,0,0,NaN,93,185,0.502703,0.503,79,136,0.581,142,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,0,0,NaN,143,275,0.520000,0.520,76,119,0.639,102,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,128,300,0.426667,336,711,0.472574,0.522,274,336,0.815,32,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,0,1,0.000000,136,248,0.548387,0.546,56,67,0.836,94,183,277,40,23,46,63,187,328,2013-2014,2013


In [2]:
# We can use the principle of euclidean distance to find the most similar NBA players to Lebron James.
# Select Lebron James from our dataset
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]

# Choose only the numeric columns (we'll use these to compute euclidean distance)
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']


In [3]:
def euclidean_distance(row):
    """
    A simple euclidean distance function
    """
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)

# Find the distance from each player in the dataset to lebron.
lebron_distance = nba.apply(euclidean_distance, axis=1)

In [4]:
# Select only the numeric columns from the NBA dataset
nba_numeric = nba[distance_columns]
# Normalize all of the numeric columns
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

In [6]:
from scipy.spatial import distance

# Fill in NA values in nba_normalized
nba_normalized.fillna(0, inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

# Create a new dataframe with distances.
distance_frame = pd.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort_values("dist", inplace=True)
# Find the most similar player to lebron (the lowest distance to lebron is lebron, the second smallest is the most similar non-lebron player)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]

In [7]:
import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)

# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(nba)/3)

# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]

# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]

In [10]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
# The column that we want to predict.
y_column = ["pts"]

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])
print(predictions)

[[6.4480e+02]
 [2.3400e+01]
 [3.3680e+02]
 [2.1740e+02]
 [9.9400e+01]
 [1.4234e+03]
 [1.0760e+02]
 [2.2400e+01]
 [1.2648e+03]
 [7.7680e+02]
 [6.9820e+02]
 [3.5020e+02]
 [1.7454e+03]
 [1.2802e+03]
 [2.6460e+02]
 [6.2940e+02]
 [4.4760e+02]
 [1.0000e+01]
 [5.5340e+02]
 [6.5460e+02]
 [1.6000e+00]
 [2.7040e+02]
 [1.1570e+03]
 [1.1940e+02]
 [1.1516e+03]
 [1.1310e+03]
 [2.9520e+02]
 [1.4638e+03]
 [3.8400e+01]
 [2.0980e+02]
 [5.8000e+00]
 [2.6220e+02]
 [4.5080e+02]
 [1.2800e+01]
 [3.3800e+01]
 [2.3800e+01]
 [2.4000e+00]
 [2.2260e+02]
 [3.8220e+02]
 [8.6800e+01]
 [1.2320e+03]
 [1.4000e+00]
 [3.4920e+02]
 [1.6400e+02]
 [1.6000e+00]
 [3.7160e+02]
 [5.9080e+02]
 [2.0980e+02]
 [6.0660e+02]
 [8.9000e+01]
 [2.6160e+02]
 [5.9380e+02]
 [6.1540e+02]
 [8.8320e+02]
 [3.4800e+01]
 [5.6820e+02]
 [7.3960e+02]
 [5.2600e+01]
 [1.7940e+02]
 [1.0382e+03]
 [3.2800e+02]
 [3.8140e+02]
 [2.9700e+02]
 [5.2000e+01]
 [7.9320e+02]
 [1.0800e+01]
 [7.9740e+02]
 [3.3640e+02]
 [1.5880e+02]
 [3.0040e+02]
 [1.5580e+03]
 [1.78

In [11]:
# Get the actual values for the test set.
actual = test[y_column]
print(actual)
# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)
print(mse)

      pts
430   703
455    12
19    352
269   302
165   104
365  1372
435   115
257    15
345  1226
124   717
259   665
379   350
225  2089
41   1249
252   226
465   625
138   401
339     3
311   499
95    527
90      0
409   159
215  1227
375   159
408  1080
20   1107
399   261
402  1264
327    26
438   144
..    ...
199   432
198  1081
72   1089
395    54
346   555
396   110
303   200
274   629
227    84
75    136
374   350
121   339
260    76
67     63
97    911
251   448
68     83
81    470
243    14
50    378
220   760
114   144
412  1012
153   174
479   490
389    29
174   618
10    572
437   146
400  1071

[159 rows x 1 columns]
pts    4135.843774
dtype: float64
